In [1]:
import random
import json
import pickle
import numpy as np
import nltk

from nltk.stem import WordNetLemmatizer
from keras.models import load_model

In [2]:
# Initialize the WordNetLemmatizer for lemmatization
lemmatizer = WordNetLemmatizer()

# Load the initial intents data from a JSON file
intents = json.load(open('michade_farms.json'))

# Load the previously pickled words and classes lists
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained model
model = load_model("my_model.keras")


In [3]:
# Define the ChatContext class to manage conversation context
class ChatContext:
    def __init__(self):
        self.current_intent = None
        self.user_input_history = []
        self.last_response = None
        self.user_name = None
        self.user_location = None
        self.user_preferences = {}
        # Add more attributes as needed to store conversation context


In [4]:
# Function to handle fallback responses when the chatbot doesn't understand the user
def handle_fallback_response(context):
    print("Mich Bot: Sorry, I don't understand. Would you like to talk to the support team?")
    while True:
        user_response = input(f"{context.user_name}: ").lower()
        if user_response in ["yes", "yeah", "ok", "sure"]:
            # Provide contact information for the support team
            support_email = "info@michadefarms.com.ng"
            support_phone = "+2348037149761"
            print(f"Mich Bot: Sure! You can reach our support team via email at {support_email} "
                  f"or by phone at {support_phone}.")
            break  # Exit the loop once a valid response is received
        elif user_response in ["no", "nope"]:
            print("Mich Bot: No problem. Would you like to ask me any other questions?")
            break  # Exit the loop once a valid response is received
        else:
            print("Mich Bot: I'm sorry, please respond with 'yes' or 'no'.")



In [5]:
# Function to prompt user for their name
def prompt_for_name():
    print("Mich Bot: Hi there! I'm Mich Bot. What's your name?")
    name = input("You: ")
    return name

In [6]:
# Function to clean up a sentence by tokenizing and lemmatizing the words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [7]:
# Function to convert a sentence into a bag of words representation
def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [8]:
# Function to predict the class (intent) of a given sentence using the trained model
def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
    return return_list

In [9]:
# Function to get a random response for a given intent from the intents data
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

In [10]:
# Prompt for the user's name
user_name = prompt_for_name()

print("Enter quit or Exit to stop the Bot.")
print(f"Mich Bot: Welcome, {user_name}! How can I assist you today?")

# Create an instance of the ChatContext class
context = ChatContext()

while True:
    message = input(f"{user_name}: ")

    # Check if the user wants to quit
    if message.lower() in ["quit", "exit"]:
        print("Mich Bot: Goodbye! Chatbot is now exiting.")
        break

    # Predict the intent and get the response
    ints = predict_class(message)
    
    # Identify gibberish responses
    gibberish_threshold = 0.1  # Adjust the threshold as needed
    if not ints or float(ints[0]['probability']) < gibberish_threshold:
        handle_gibberish_response()
        continue  # Skip the rest of the loop for this iteration
        
   # Identify fallback responses
    fallback_threshold = 0.2  # Adjust the threshold as needed
    if float(ints[0]['probability']) < fallback_threshold:
        handle_fallback_response()
        continue  # Skip the rest of the loop for this iteration
    
    res = get_response(ints, intents)
    

    # Update the context with the latest user input and intent
    context.current_intent = ints[0]['intent']
    context.user_input_history.append(message)

    # Update last_response with the current response
    context.last_response = res

    # Check if the user provided their name or location
    if context.current_intent == 'greeting' and 'user_name' in message.lower():
        context.user_name = message
    elif context.current_intent == 'location':
        context.user_location = message
        
     # Handle fallback responses when the chatbot doesn't understand the user
    if context.current_intent == 'fallback':
        handle_fallback_response(context)
        continue  # Skip the rest of the loop for this iteration

    # Handle fallback responses when the chatbot doesn't understand the user
    if context.current_intent == 'fallback' and context.last_response.lower() == "hello":
        handle_gibberish_response()
        continue  # Skip the rest of the loop for this iteration

    print("Mich Bot:", res)

Mich Bot: Hi there! I'm Mich Bot. What's your name?
You: Dawn
Enter quit or Exit to stop the Bot.
Mich Bot: Welcome, Dawn! How can I assist you today?
Dawn: farm is where?
1/1 [==============================] - 0s 124ms/step
Mich Bot: MichAde Farms is located at along Osogbo-Ede-Iwo-Ibadan Express way, Ede Osun State.
Dawn: what do you do
1/1 [==============================] - 0s 24ms/step
Mich Bot: Sorry, I don't understand. Would you like to talk to the support team?
None: what does the farm do
Mich Bot: I'm sorry, please respond with 'yes' or 'no'.
None: yes
Mich Bot: Sure! You can reach our support team via email at info@michadefarms.com.ng or by phone at +2348037149761.
Dawn: what is michade farms about
1/1 [==============================] - 0s 25ms/step
Mich Bot: MichAde Farms is a ranch founded on the 6th of June 2016 as a family business.It is located along Osogbo-Ede-Iwo-Ibadan Express way Osun State,was established with the love for healthy living, empowerment to young people

In [11]:
# import random
# import numpy as np
# import json
# import pickle
# import tensorflow as tf
# import nltk
# from nltk.stem import WordNetLemmatizer
# from keras.models import load_model

# # Constants
# GIBBERISH_THRESHOLD = 0.1
# FALLBACK_THRESHOLD = 0.2

# # Initialize the WordNetLemmatizer for lemmatization
# lemmatizer = WordNetLemmatizer()

# # # Load the initial intents data from a JSON file
# intents = json.load(open('michade_farms.json'))

# # Load the previously pickled words and classes lists
# words = pickle.load(open('words.pkl', 'rb'))
# classes = pickle.load(open('classes.pkl', 'rb'))

from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

# # Load the trained model
# try:
#     model = load_model("my_model.keras")
# except Exception as e:
#     print(f"Error loading the model: {e}")
#     # Handle the error or exit the program

# # Define the ChatContext class to manage conversation context
# class ChatContext:
#     def __init__(self):
#         self.current_intent = None
#         self.user_input_history = []
#         self.last_response = None
#         self.user_name = None
#         self.user_location = None
#         self.user_preferences = {}
#         # Add more attributes as needed to store conversation context

# # Function to handle gibberish responses when the chatbot doesn't understand the user
# def handle_gibberish_response():
#     print("Mich Bot: I'm sorry, but I couldn't understand your input. Please try again.")

# # Function to handle fallback responses when the chatbot doesn't understand the user
# def handle_fallback_response(context):
#     print("Mich Bot: Sorry, I don't understand. Would you like to talk to the support team?")
#     while True:
#         user_response = input(f"{context.user_name}: ").lower()
#         if user_response in ["yes", "yeah", "ok", "sure"]:
#             # Provide contact information for the support team
#             support_email = "info@michadefarms.com.ng"
#             support_phone = "+2348037149761"
#             print(f"Mich Bot: Sure! You can reach our support team via email at {support_email} "
#                   f"or by phone at {support_phone}.")
#             break  # Exit the loop once a valid response is received
#         elif user_response in ["no", "nope"]:
#             print("Mich Bot: No problem. Would you like to ask me any other questions?")
#             break  # Exit the loop once a valid response is received
#         else:
#             print("Mich Bot: I'm sorry, please respond with 'yes' or 'no'.")
    
#     # Update context with the fact that the chatbot didn't understand the user's input
#     context.last_response = "I don't understand. Please try again."


# # Function to prompt user for their name
# def prompt_for_name():
#     print("Mich Bot: Hi there! I'm Mich Bot. What's your name?")
#     name = input("You: ")
#     return name

# # Function to clean up a sentence by tokenizing and lemmatizing the words
# def clean_up_sentence(sentence):
#     sentence_words = nltk.word_tokenize(sentence)
#     sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
#     return sentence_words

# # Function to convert a sentence into a bag of words representation
# def bag_of_words(sentence):
#     sentence_words = clean_up_sentence(sentence)
#     bag = [0] * len(words)
#     for w in sentence_words:
#         for i, word in enumerate(words):
#             if word == w:
#                 bag[i] = 1
#     return np.array(bag)

# # Function to predict the class (intent) of a given sentence using the trained model
# def predict_class(sentence):
#     bow = bag_of_words(sentence)
#     res = model.predict(np.array([bow]))[0]
#     ERROR_THRESHOLD = 0.25
#     results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
#     return return_list

# # Debugging output: Print some predictions
# sample_questions = ["Is Michade Farms a farm?", "Are there any health-conscious options?", "Are there any specific hours for customer service?"]
# for question in sample_questions:
#     ints = predict_class(question)
#     print(f"Question: {question}, Predicted Intents: {ints}")

# # Function to get a random response for a given intent from the intents data
# def get_response(intents_list, intents_json):
#     tag = intents_list[0]['intent']
#     list_of_intents = intents_json['intents']
#     for i in list_of_intents:
#         if i['tag'] == tag:
#             result = random.choice(i['responses'])
#             break
#     return result

# # Prompt for the user's name
# user_name = prompt_for_name()

# print("Enter quit or Exit to stop the Bot.")
# print(f"Mich Bot: Welcome, {user_name}! How can I assist you today?")

# # Create an instance of the ChatContext class
# context = ChatContext()

# while True:
#     message = input(f"{user_name}: ")

#     # Check if the user wants to quit
#     if message.lower() in ["quit", "exit"]:
#         print("Mich Bot: Goodbye! Chatbot is now exiting.")
#         break

#     # Predict the intent and get the response
#     ints = predict_class(message)
    
#     # Debugging output
#     print("Predicted Intents:", ints)
    
#     # Identify gibberish responses
#     if not ints or float(ints[0]['probability']) < GIBBERISH_THRESHOLD:
#         handle_gibberish_response()
#         continue  # Skip the rest of the loop for this iteration
        
#     # Identify fallback responses
#     if float(ints[0]['probability']) < FALLBACK_THRESHOLD:
#         handle_fallback_response(context)
#         continue  # Skip the rest of the loop for this iteration
    
#     res = get_response(ints, intents)

#     # Update the context with the latest user input and intent
#     context.current_intent = ints[0]['intent']
#     context.user_input_history.append(message)

#     # Update last_response with the current response
#     context.last_response = res

#     # Check if the user provided their name or location
#     if context.current_intent == 'greeting' and 'user_name' in message.lower():
#         context.user_name = message
#     elif context.current_intent == 'location':
#         context.user_location = message

#     # Handle fallback responses when the chatbot doesn't understand the user
#     if context.current_intent == 'fallback':
#         handle_fallback_response(context)
#         continue  # Skip the rest of the loop for this iteration

#     # Handle fallback responses when the chatbot doesn't understand the user
#     if context.current_intent == 'fallback' and context.last_response.lower() == "hello":
#         handle_gibberish_response()
#         continue  # Skip the rest of the loop for this iteration

#     print("Mich Bot:", res)


In [ ]:
from tkinter import *
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

# Constants and setup for the chatbot
GIBBERISH_THRESHOLD = 0.1
FALLBACK_THRESHOLD = 0.2

bot_name = "Mich Bot"

# Initialize the WordNetLemmatizer for lemmatization
lemmatizer = WordNetLemmatizer()

# Load the initial intents data from a JSON file
intents = json.load(open('michade_farms.json'))

# Load the previously pickled words and classes lists
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained model
try:
    model = load_model("my_model.keras")
except Exception as e:
    print(f"Error loading the model: {e}")
    # Handle the error or exit the program

class ChatContext:
    def __init__(self):
        self.current_intent = None
        self.user_input_history = []
        self.last_response = None
        self.user_name = None
        self.user_location = None
        self.user_preferences = {}

# Function to handle gibberish responses when the chatbot doesn't understand the user
def handle_gibberish_response():
    return "Mich Bot: I'm sorry, but I couldn't understand your input. Please try again."

# Function to handle fallback responses when the chatbot doesn't understand the user
def handle_fallback_response(context):
    response = "Mich Bot: Sorry, I don't understand. Would you like to talk to the support team?"
    user_response = input(f"{context.user_name}: ").lower()
    if user_response in ["yes", "yeah", "ok", "sure"]:
        # Provide contact information for the support team
        support_email = "info@michadefarms.com.ng"
        support_phone = "+2348037149761"
        response = (f"Mich Bot: Sure! You can reach our support team via email at {support_email} "
                    f"or by phone at {support_phone}.")
    elif user_response in ["no", "nope"]:
        response = "Mich Bot: No problem. Would you like to ask me any other questions?"
    else:
        response = "Mich Bot: I'm sorry, please respond with 'yes' or 'no'."
    
    # Update context with the fact that the chatbot didn't understand the user's input
    context.last_response = "I don't understand. Please try again."
    return response

# Function to clean up a sentence by tokenizing and lemmatizing the words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

# Function to convert a sentence into a bag of words representation
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict the class (intent) of a given sentence using the trained model
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

# Function to get a random response for a given intent from the intents data
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result